In [21]:
import pandas as pd

In [22]:
train_file = pd.read_csv('iot-datasets/train.csv')
severity_type = pd.read_csv('iot-datasets/severity_type.csv')

In [27]:
def check_unique_ids(dataset):
    unique_ids = dataset['id'].nunique()
    repeated_ids = len(dataset['id']) - unique_ids
    print(f"Number of unique ids: {unique_ids}")
    print(f"Number of ids repeated more than once: {repeated_ids}")

check_unique_ids(train_file)
check_unique_ids(severity_type)

Number of unique ids: 7381
Number of ids repeated more than once: 0
Number of unique ids: 18552
Number of ids repeated more than once: 0
